# Library

In [2]:
import warnings

warnings.filterwarnings("ignore", message="Boolean Series key will be reindexed to match DataFrame index")
warnings.filterwarnings("ignore", category=FutureWarning, module="pyntcloud.io.ply")
warnings.filterwarnings("ignore", category=RuntimeWarning)

import os
import time
import itertools
import shutil
import subprocess

import pandas as pd
import numpy as np

from scipy.spatial import ConvexHull
from pyntcloud import PyntCloud

from vggsfm.datasets.imc_helper import (
        read_cameras_binary, write_cameras_text,
        read_images_binary, write_images_text,
        read_points3D_binary, write_points3D_text
    )

from Utilities.utilities import read_points3D, write_ply, calculate_volume, calculate_volume_file

# Calculate Volume

In [26]:
def get_folders_in_directory(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    return folders

def list_directory_contents(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    return items

def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Get the list of folders in the directory
# directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3'
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_100_5'
folders = get_folders_in_directory(directory_path)
folders.sort()

# print(folders)

##################################################
# List Algorithm
##################################################
list_algorithm = [
    'MVG',
    'MVG_MVS',
    # 'COLMAP',
    # 'COLMAP_MVS',
    # 'VGGSFM', # Use Python 3.12.7
    # 'MVE',
]

##################################################
# Run No Background
##################################################
list_bg = [
    'with_bg', 
    # 'no_bg',
    ]

##################################################
# Scale and Frame
##################################################
# list_scale_percent = [100, 50]
# list_scale_percent = [50]
# list_scale_percent = [50]
# list_max_frames = [30, 45, 60, 75, 90, 105]
list_scale_percent = [100]
list_max_frames = [5]


# list_subject_testing = ['8', '9', '17', '19', '22', '67']
# list_subject_testing = [1, 2, 6, 7, 8, 9, 
#                         10, 11, 12, 16, 17, 18, 19, 
#                         21, 22, 23, 24, 27, 29, 
#                         30, 35, 
#                         41, 45, 46, 49, 
#                         51, 55, 56, 59, 
#                         60, 61, 63, 67, 69]
# list_subject_testing = [str(x) for x in list_subject_testing]

##################################################
# Run
##################################################
results_volume = []


for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    for (algorithm, bg) in itertools.product(list_algorithm, list_bg):
        # folders = ['2']
        for folder in folders:
            folder_path = os.path.join(directory_path, folder)
            
            # Get the list of files and folders in the directory
            contents = list_directory_contents(folder_path)
            contents = [x for x in contents if ':Zone.Identifier' not in x]
            subject = folder_path.split('/')[-1].split('.')[0]

            # if subject in list_subject_testing:
            if True:

                for data in contents:         
                    print('')       
                    print(f'Subject {subject} - {data} - {bg} - {algorithm} - Scale: {scale_percent}, Max Frames: {max_frames}')
                    
                    # If no bg
                    if bg == 'with_bg':
                        path_bg = ''
                    else:
                        path_bg = '_no_bg'
                        
                    start_time = time.time()
                    error = ''
                    
                    path_base = f'/home/weiyanpeh/Git/SFM_Related/CADENCE'

                    if "COLMAP" in algorithm:
                        path_volume = f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm/dense/sparse"
                        points3D = read_points3D_binary(path_volume + "/points3D.bin")

                        write_points3D_text(points3D, path_volume + "/points3D.txt")

                        points3D_file = path_volume + "/points3D.txt"
                        output_ply_file = path_volume + "/cloud_and_poses.ply"
                        
                        points3D = read_points3D(points3D_file)
                        write_ply(points3D, output_ply_file)

                        print(f'PLY file saved to {output_ply_file}')


                    dict_algorithm_path = {
                        'MVG': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/MVG_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm", 
                            'file_name': ["cloud_and_poses.ply"]},
                        
                        'MVG_MVS': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/MVG_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/mvs", 
                            'file_name': ["scene_dense_mesh.ply", "scene_dense_texture.ply", "scene_dense_mesh_refine.ply"]},
                            # "scene_dense.ply" always get error "field 'n_coords' occurs more than once"
                        
                        'COLMAP_MVS': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/mvs", 
                            'file_name': ["scene_dense_mesh.ply", "scene_dense_texture.ply", "scene_dense_mesh_refine.ply"]},
                            # "scene_dense.ply" always get error "field 'n_coords' occurs more than once"

                        # OpenMVS Colamp
                        'COLMAP': {
                            'file_path': f"{path_base}/SHAPE_Frames_output/COLMAP_MVS_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sfm/dense/sparse", 
                            'file_name': ["cloud_and_poses.ply"]},
                        
                        # 'MVG': {
                        #     'file_path': f"{path_base}/SHAPE_Frames_output/MVG/{subject}/{data}/reconstruction_sequential",
                        #     'file_name': ["cloud_and_poses.ply"]},
                        
                        # Real COLMAP
                        # 'COLMAP': {
                            # 'file_path': f"{path_base}/SHAPE_Frames_input/images_input_colmap_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/dense", 
                            # 'file_name': ["fused.ply", "meshed-poisson.ply", "meshed-delaunay.ply"]},
                        
                        'VGGSFM': {
                            'file_path': f"{path_base}/SHAPE_Frames_input/images_input_vggsfm_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/sparse", 
                            'file_name': ["output.ply"]},

                        'MVE': {
                            'file_path': f"{path_base}/SHAPE_Frames_input/images_input_mve_{scale_percent}_{max_frames}{path_bg}/{subject}/{data}/convert", 
                            'file_name': ["pset-L2.ply", "surface-L2.ply", "surface-L2-clean.ply"]},

                    }

                    file_path = dict_algorithm_path[algorithm]['file_path']
                    list_file_name = dict_algorithm_path[algorithm]['file_name']
                    sfm_model = ''
                    
                    if algorithm in ['MVG', 'MVG_MVS']:
                        sfm_model = 'MVG_MVS'
                    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
                        sfm_model = 'COLMAP_MVS'
                    else:
                        sfm_model = algorithm

                    for file_name in list_file_name:
                        error = ''
                        try:
                            start_time = time.time()  # Record the start time

                            print(file_path)
                            print(file_name)

                            volume = calculate_volume_file(
                                file_path = file_path, 
                                file_name = file_name
                                )
                            
                            print(f'{file_name} - Volume: {round(volume, 3)} unit3')
                            
                            end_time = time.time()  # Record the end time
                            execution_time = end_time - start_time  # Calculate the execution 
                            
                            results_volume.append([
                                subject, data, sfm_model,
                                algorithm, bg, scale_percent, max_frames,
                                execution_time, 'PyntCloud', file_path, file_name, 
                                volume, error])
                            
                        except Exception as error:
                            print(file_name, error)
                            end_time = time.time()  # Record the end time
                            execution_time = end_time - start_time  # Calculate the execution 
                            results_volume.append([
                                subject, data, sfm_model,
                                algorithm, bg, scale_percent, max_frames, 
                                execution_time, 'PyntCloud', file_path, file_name, 
                                0, error])

# Convert to DataFrame
columns = ['Subject', 'Image Data', 'SFM',
           'Algorithm', 'BG', 'Scale', 'Max Frames', 
           'Execution_time', 'Volume Calculation', 'File_Path', 'File_Name', 
           'Volume', 'Error']
df_results = pd.DataFrame(results_volume, columns=columns)
df_results = df_results.drop('Execution_time', axis=1)
df_results.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume.csv', index=False)
    



Subject 71 - 71D0R - with_bg - MVG - Scale: 100, Max Frames: 5
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_output/MVG_MVS_100_5/71/71D0R/sfm
cloud_and_poses.ply
cloud_and_poses.ply - Volume: 0.432 unit3

Subject 71 - 71D0L - with_bg - MVG - Scale: 100, Max Frames: 5
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_output/MVG_MVS_100_5/71/71D0L/sfm
cloud_and_poses.ply
cloud_and_poses.ply QH6214 qhull input error: not enough points(3) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 825214313  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0


Subject 71 - 71D1R - with_bg - MVG - Scale: 100, Max Frames: 5
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_output/MVG_MVS_100_5/71/71D1R/sfm
cloud_and_poses.ply
cloud_and_poses.ply No points given

Subject 71 - 71D1L - with_bg - MVG - Scale: 100, Max Frames: 5
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_output/MVG_MVS_100

In [27]:
subject_sorted = list(set(df_results['Subject'].tolist()))
subject_sorted.sort()
subject_sorted

['71', '72', '73', '74']

In [28]:
df_results

,Subject,Image Data,SFM,Algorithm,BG,Scale,Max Frames,Volume Calculation,File_Path,File_Name,Volume,Error
0,71,71D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.431691,
1,71,71D0L,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,QH6214 qhull input error: not enough points(3)...
2,71,71D1R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,No points given
3,71,71D1L,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
4,72,72D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
5,73,73D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,74.572982,
6,74,74D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,2268.287191,
7,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
8,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_texture.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
9,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh_refine.ply,0.000000,[Errno 2] No such file or directory: '/home/we...


In [29]:
df_results_grouped = df_results.copy()

df_results_grouped = df_results_grouped[df_results_grouped['Volume'] != 0].reset_index(drop=True)
df_results_grouped = df_results_grouped.drop(['BG', 'Scale', 'Max Frames'], axis=1)

# Group by Subject, Image Data, and Algorithm
df_results_grouped = df_results_grouped.groupby(['Subject', 'Image Data', 'Algorithm', 'File_Name']).sum()

# Check if Volume is non-zero
df_results_grouped['!0'] = df_results_grouped['Volume'] != 0
df_results_grouped.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume_grouped.csv')

print(df_results_grouped)

                                                               Volume    !0
Subject Image Data Algorithm File_Name                                     
71      71D0R      MVG       cloud_and_poses.ply             0.431691  True
73      73D0R      MVG       cloud_and_poses.ply            74.572982  True
74      74D0R      MVG       cloud_and_poses.ply          2268.287191  True
                   MVG_MVS   scene_dense_mesh.ply          155.341813  True
                             scene_dense_mesh_refine.ply   154.125619  True
                             scene_dense_texture.ply       154.125191  True


/tmp/ipykernel_21940/3151457314.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_results_grouped = df_results_grouped.groupby(['Subject', 'Image Data', 'Algorithm', 'File_Name']).sum()


In [30]:
len(set(list(df_results['Image Data'].tolist())))

7

In [31]:
list_unique_images = list(set(df_results['Image Data'].tolist()))
print('Unique Images:', len(list_unique_images))
n = list_unique_images

list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']

for algorithm in list_algorithm:
    df_temp = df_results[(df_results['Algorithm'] == algorithm)].reset_index(drop=True)
    df_temp = df_temp[df_temp['Volume'] != 0].reset_index(drop=True)
    
    if algorithm in ['MVG', 'COLMAP']:
        df_temp = df_temp[(df_temp['File_Name'] == 'cloud_and_poses.ply')].reset_index(drop=True)
        
        print('')
        print('Algorithm:', algorithm)
        print('Total number of videos:', len(df_temp))
        
    else:
        list_mesh = ['scene_dense_mesh.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_texture.ply']
        
        for mesh in list_mesh:
            df_temp_mesh = df_temp[(df_temp['File_Name'] == mesh)].reset_index(drop=True)
            
            print('')
            print('Algorithm:', algorithm, '-', mesh)
            print('Total number of videos:', len(df_temp_mesh))

Unique Images: 7

Algorithm: MVG
Total number of videos: 3

Algorithm: MVG_MVS - scene_dense_mesh.ply
Total number of videos: 1

Algorithm: MVG_MVS - scene_dense_mesh_refine.ply
Total number of videos: 1

Algorithm: MVG_MVS - scene_dense_texture.ply
Total number of videos: 1

Algorithm: COLMAP
Total number of videos: 0

Algorithm: COLMAP_MVS - scene_dense_mesh.ply
Total number of videos: 0

Algorithm: COLMAP_MVS - scene_dense_mesh_refine.ply
Total number of videos: 0

Algorithm: COLMAP_MVS - scene_dense_texture.ply
Total number of videos: 0


In [32]:
list_subject_testing = list(set(df_results['Subject'].tolist()))
list_subject_testing = [int(x) for x in list_subject_testing]
list_subject_testing.sort()
list_subject_testing = [str(x) for x in list_subject_testing]
# list_subject_testing = ['8', '9', '17', '19', '22', '67']

# list_background = ['with_bg', 'no_bg']
# list_scale_percent = [100, 50]

list_background = ['with_bg']
list_scale_percent = [50]
list_max_frames = [3]
df_results_save = []



list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']

for algorithm in list_algorithm:

    if algorithm in ['MVG', 'COLMAP']:
        list_mesh = ['cloud_and_poses.ply']
    else:
        list_mesh = ['scene_dense_mesh.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_texture.ply']
    
    if algorithm in ['MVG', 'MVG_MVS']:
        sfm_model = 'MVG_MVS'
    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
        sfm_model = 'COLMAP_MVS'
    else:
        sfm_model = algorithm
    
    for file_name in list_mesh:
    
        df_test = df_results[
            (df_results['Algorithm'] == algorithm) &
            (df_results['File_Name'] == file_name)]
        
        total = len(df_test)
        fail = len(df_test[df_test['Volume'] == 0])
        success = total - fail
        try:
            success_rate = round(success/total*100, 1)
        except:
            success_rate = 0
    
        # Calculate success rate per subject
        list_sr_100 = []
        list_sr_99_70 = []
        list_sr_69_1 = []
        list_sr_0 = []

        print('')
        print('#'*60)
        print(f'Background: {bg}, Scale: {scale_percent}, Max Frames: {max_frames}')
        print(algorithm, '-', file_name)
        print('Total:', total)
        print('Success:', success)
        print('Fail:', fail)
        print('SR%:', success_rate, '%')
        print('')

        for subject in list_subject_testing:

            df_results_subject = df_results[
                (df_results['Algorithm'] == algorithm) &
                (df_results['Subject'] == subject) &
                (df_results['File_Name'] == file_name)]
            
            list_images = df_results_subject['Image Data'].tolist()
            list_volume = df_results_subject['Volume'].tolist()
            list_volume_zero = [x for x in list_volume if x == 0]

            subject_total = len(list_volume)
            subject_fail = len(list_volume_zero)
            subject_success = subject_total - subject_fail
            try:
                subject_success_rate = subject_success/subject_total*100
            except:
                subject_success_rate = 0
            
            if subject_success_rate == 100:
                list_sr_100.append(subject)
            elif (subject_success_rate <= 99) and (subject_success_rate >= 70):
                list_sr_99_70.append(subject)
            elif (subject_success_rate <= 69) and (subject_success_rate >= 1):
                list_sr_69_1.append(subject)
            else:
                list_sr_0.append(subject)
                
            # print(f'Subject {subject}: {subject_success_rate}')
            if subject_success_rate == 100:
                print(f'Subject {subject}: {list_images}')

        print('')
        print(f'100%  : {list_sr_100} - {len(list_sr_100)}')
        print(f'99-70%: {list_sr_99_70} - {len(list_sr_99_70)}')
        print(f'69-1% : {list_sr_69_1} - {len(list_sr_69_1)}')
        print(f'0%    : {list_sr_0} - {len(list_sr_0)}')

        list_sr_100 = ', '.join(list_sr_100)
        list_sr_99_70 = ', '.join(list_sr_99_70)
        list_sr_69_1 = ', '.join(list_sr_69_1)
        list_sr_0 = ', '.join(list_sr_0)

        results = [sfm_model, algorithm, bg, scale_percent, max_frames, file_name, 
                    total, success, fail, success_rate,
                    list_sr_100, list_sr_99_70, list_sr_69_1, list_sr_0]
        df_results_save.append(results)
    
columns = ['SFM', 'Algorithm', 'Background', 'Scale', 'Max Frame', 'File',
           'Total', 'Success', 'Fail', 'Success Rate',
           '100', '99-70', '69-1', '0']
df_results_save = pd.DataFrame(df_results_save, columns=columns)
df_results_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_volume_summary.csv', index=False)


############################################################
Background: with_bg, Scale: 100, Max Frames: 5
MVG - cloud_and_poses.ply
Total: 7
Success: 3
Fail: 4
SR%: 42.9 %

Subject 73: ['73D0R']
Subject 74: ['74D0R']

100%  : ['73', '74'] - 2
99-70%: [] - 0
69-1% : ['71'] - 1
0%    : ['72'] - 1

############################################################
Background: with_bg, Scale: 100, Max Frames: 5
MVG_MVS - scene_dense_mesh.ply
Total: 7
Success: 1
Fail: 6
SR%: 14.3 %

Subject 74: ['74D0R']

100%  : ['74'] - 1
99-70%: [] - 0
69-1% : [] - 0
0%    : ['71', '72', '73'] - 3

############################################################
Background: with_bg, Scale: 100, Max Frames: 5
MVG_MVS - scene_dense_mesh_refine.ply
Total: 7
Success: 1
Fail: 6
SR%: 14.3 %

Subject 74: ['74D0R']

100%  : ['74'] - 1
99-70%: [] - 0
69-1% : [] - 0
0%    : ['71', '72', '73'] - 3

############################################################
Background: with_bg, Scale: 100, Max Frames: 5
MVG_MVS - scene_d

In [33]:
df_results_save

,SFM,Algorithm,Background,Scale,Max Frame,File,Total,Success,Fail,Success Rate,100,99-70,69-1,0
0,MVG_MVS,MVG,with_bg,100,5,cloud_and_poses.ply,7,3,4,42.9,"73, 74",,71,72
1,MVG_MVS,MVG_MVS,with_bg,100,5,scene_dense_mesh.ply,7,1,6,14.3,74,,,"71, 72, 73"
2,MVG_MVS,MVG_MVS,with_bg,100,5,scene_dense_mesh_refine.ply,7,1,6,14.3,74,,,"71, 72, 73"
3,MVG_MVS,MVG_MVS,with_bg,100,5,scene_dense_texture.ply,7,1,6,14.3,74,,,"71, 72, 73"
4,COLMAP_MVS,COLMAP,with_bg,100,5,cloud_and_poses.ply,0,0,0,0.0,,,,"71, 72, 73, 74"
5,COLMAP_MVS,COLMAP_MVS,with_bg,100,5,scene_dense_mesh.ply,0,0,0,0.0,,,,"71, 72, 73, 74"
6,COLMAP_MVS,COLMAP_MVS,with_bg,100,5,scene_dense_mesh_refine.ply,0,0,0,0.0,,,,"71, 72, 73, 74"
7,COLMAP_MVS,COLMAP_MVS,with_bg,100,5,scene_dense_texture.ply,0,0,0,0.0,,,,"71, 72, 73, 74"


In [34]:
algorithm

'COLMAP_MVS'

In [35]:
list_subject_testing = list(set(df_results['Subject'].tolist()))
list_subject_testing = [int(x) for x in list_subject_testing]
list_subject_testing.sort()
list_subject_testing = [str(x) for x in list_subject_testing]

# list_algorithm = ['MVG', 'MVG_MVS', 'COLMAP', 'COLMAP_MVS']
# list_sfm_model = ['MVG_MVS', 'COLAMP_MVS']

list_algorithm = ['MVG', 'MVG_MVS']
list_sfm_model = ['MVG_MVS']

df_results_save = []

for algorithm in list_algorithm:
    
    if algorithm in ['MVG', 'MVG_MVS']:
        sfm_model = 'MVG_MVS'
    elif algorithm in ['COLMAP', 'COLMAP_MVS']:
        sfm_model = 'COLMAP_MVS'
    else:
        sfm_model = algorithm
    
    if algorithm in ['MVG', 'COLMAP']:
        list_file_name = ['cloud_and_poses.ply']
    elif algorithm in ['MVG_MVS', 'COLMAP_MVS']:
        list_file_name = ['scene_dense_texture.ply', 'scene_dense_mesh_refine.ply', 'scene_dense_mesh.ply']
        
    for file_name in list_file_name:
        
        for subject in list_subject_testing:

            df_results_subject = df_results[(df_results['Subject'] == subject)]    
            df_results_subject = df_results_subject[
                                                    (df_results_subject['Algorithm'] == algorithm) &
                                                    (df_results_subject['BG'] == 'with_bg') &
                                                    (df_results_subject['Scale'] == 50) &
                                                    (df_results_subject['Max Frames'] == 3) &
                                                    (df_results_subject['File_Name'] == file_name)]    
            
            list_images = df_results_subject['Image Data'].tolist()
            list_volume = df_results_subject['Volume'].tolist()
            list_volume_zero = [x for x in list_volume if x == 0]
            
            total = len(list_volume)
            fail = len(list_volume_zero)
            success = total - fail
            
            try:
                success_rate = success/total*100
            except:
                success_rate = 0 
            
            results = [algorithm, sfm_model, file_name, subject, ', '.join(list_images), total, fail, success, success_rate]
            df_results_save.append(results)
            
            print(f'Subject {subject}: {success_rate}')

columns = ['Algorithm', 'SFM', 'File_Name', 'Subject', 'Image Data', 'Total', 'Fail', 'Success', 'Success_rate']
df_results_save = pd.DataFrame(df_results_save, columns=columns)
df_results_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_SFM_subject_success_rate.csv', index=False)

def print_specific_data(df, title):
    print(f'{title}: {len(df)} -> {", ".join(df['Subject'].tolist())}')
    return None


for algorithm in list_algorithm:
    print('')
    print(algorithm)
    df_temp = df_results_save[df_results_save['Algorithm'] == algorithm]
    if algorithm in ['MVG_MVS', 'COLMAP_MVS']:
        df_temp = df_results_save[df_results_save['File_Name'] == 'scene_dense_mesh.ply']
    print_specific_data(df_temp[(df_temp['Success_rate'] == 100)], '100%')
    print_specific_data(df_temp[(df_temp['Success_rate'] < 100) & (df_temp['Success_rate'] >= 70)], '99-70%')
    print_specific_data(df_temp[(df_temp['Success_rate'] < 70) & (df_temp['Success_rate'] >= 50)], '69-50%')
    print_specific_data(df_temp[(df_temp['Success_rate'] <= 49) & (df_temp['Success_rate'] > 0)], '49-1%')
    print_specific_data(df_temp[(df_temp['Success_rate'] == 0)], '0%')

Subject 71: 0
Subject 72: 0
Subject 73: 0
Subject 74: 0
Subject 71: 0
Subject 72: 0
Subject 73: 0
Subject 74: 0
Subject 71: 0
Subject 72: 0
Subject 73: 0
Subject 74: 0
Subject 71: 0
Subject 72: 0
Subject 73: 0
Subject 74: 0

MVG
100%: 0 -> 
99-70%: 0 -> 
69-50%: 0 -> 
49-1%: 0 -> 
0%: 4 -> 71, 72, 73, 74

MVG_MVS
100%: 0 -> 
99-70%: 0 -> 
69-50%: 0 -> 
49-1%: 0 -> 
0%: 4 -> 71, 72, 73, 74


In [36]:
df_results_save

,Algorithm,SFM,File_Name,Subject,Image Data,Total,Fail,Success,Success_rate
0,MVG,MVG_MVS,cloud_and_poses.ply,71,,0,0,0,0
1,MVG,MVG_MVS,cloud_and_poses.ply,72,,0,0,0,0
2,MVG,MVG_MVS,cloud_and_poses.ply,73,,0,0,0,0
3,MVG,MVG_MVS,cloud_and_poses.ply,74,,0,0,0,0
4,MVG_MVS,MVG_MVS,scene_dense_texture.ply,71,,0,0,0,0
5,MVG_MVS,MVG_MVS,scene_dense_texture.ply,72,,0,0,0,0
6,MVG_MVS,MVG_MVS,scene_dense_texture.ply,73,,0,0,0,0
7,MVG_MVS,MVG_MVS,scene_dense_texture.ply,74,,0,0,0,0
8,MVG_MVS,MVG_MVS,scene_dense_mesh_refine.ply,71,,0,0,0,0
9,MVG_MVS,MVG_MVS,scene_dense_mesh_refine.ply,72,,0,0,0,0


In [37]:
list_subject_best = [1, 17, 29, 12, 21, 36, 49, 61]
list_subject_best = [str(x) for x in list_subject_best]

df_filter = df_results[df_results['Subject'].isin(list_subject_best)].reset_index(drop=True)

for i, row in df_filter.iterrows():
    subject = row['Subject']
    data = row['Image Data']
    algorithm = row['Algorithm']
    volume = round(row['Volume'], 3)
    filename = row['File_Name']
    
    print(f'Subject {subject} - {data} - {algorithm} - {filename} - {volume}')

In [38]:
df_results

,Subject,Image Data,SFM,Algorithm,BG,Scale,Max Frames,Volume Calculation,File_Path,File_Name,Volume,Error
0,71,71D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.431691,
1,71,71D0L,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,QH6214 qhull input error: not enough points(3)...
2,71,71D1R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,No points given
3,71,71D1L,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
4,72,72D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
5,73,73D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,74.572982,
6,74,74D0R,MVG_MVS,MVG,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,cloud_and_poses.ply,2268.287191,
7,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
8,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_texture.ply,0.000000,[Errno 2] No such file or directory: '/home/we...
9,71,71D0R,MVG_MVS,MVG_MVS,with_bg,100,5,PyntCloud,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,scene_dense_mesh_refine.ply,0.000000,[Errno 2] No such file or directory: '/home/we...


In [41]:
files_ply = [
    ('SFM', 'cloud_and_poses.ply'), 
    ('MVS_mesh', 'scene_dense_mesh.ply'),
    ('MVS_dense', 'scene_dense_texture.ply'),
    ('MVS_refine', 'scene_dense_mesh_refine.ply'),
    ]

df_to_save = []
# for subject in ['1', '6', '12', '17', '29', '67']:
for subject in ['74']:
    df_results_subject = df_results[df_results['Subject'] == subject].reset_index(drop=True)
    
    list_file_name = list(set(df_results_subject['Image Data'].tolist()))
    list_file_name.sort()

    
    for file_name in list_file_name:
        df_results_subject_file = df_results_subject[df_results_subject['Image Data'] == file_name].reset_index(drop=True)
        
        to_save_list = []

        for mesh_type, ply in files_ply:
            volume = df_results_subject_file[df_results_subject_file['File_Name'] == ply].reset_index(drop=True)['Volume'][0]
            file_path = df_results_subject_file[df_results_subject_file['File_Name'] == ply].reset_index(drop=True)['File_Path'][0]
            
            df_to_save.append([subject, file_name, mesh_type, os.path.join(file_path, ply), volume])
            
columns = ['Subject', 'Image Data', 'Type', 'File_Path', 'Volume']
df_to_save = pd.DataFrame(df_to_save, columns=columns)

df_to_save.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_BEST_subjects.csv', index=False)
    
    

In [42]:
df_to_save

,Subject,Image Data,Type,File_Path,Volume
0,74,74D0R,SFM,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,2268.287191
1,74,74D0R,MVS_mesh,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,155.341813
2,74,74D0R,MVS_dense,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,154.125191
3,74,74D0R,MVS_refine,/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_...,154.125619
